In [163]:
from elasticsearch import Elasticsearch, helpers
import os

# Connect to Elasticsearch
ES_HOST = "http://localhost:9200"
es = Elasticsearch([ES_HOST])

In [164]:
# Delete the index if it exists
if es.indices.exists(index='index_recipes'):
    es.indices.delete(index='index_recipes')

In [165]:
# Define the mapping for the index
mapping = {
    "mappings": {
        "properties": {
            "title": {"type": "text",
                      "analyzer": "italian",
                      "search_analyzer": "italian"},
            "content": {"type": "text", 
                        "analyzer": "italian",
                        "search_analyzer": "italian"}
        }
    }
}

es.indices.create(index='index_recipes', body=mapping)
print("Index 'index_recipes' with mapping created successfully.")

Index 'index_recipes' with mapping created successfully.


In [166]:
def bulk_index(directory, index_name):
    actions = []

    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            path = os.path.join(directory, filename)
            with open(path, "r", encoding="utf-8", errors="ignore") as f:
                text = f.read()

            # Remove file extension for title
            title = os.path.splitext(filename)[0]
            
            action = {
                "_index": index_name,
                "_source": {
                    "title": title,
                    "content": text
                }
            }
            actions.append(action)

    # Bulk index the documents with only one call to Elasticsearch
    if actions:
        helpers.bulk(es, actions)
        print(f"{len(actions)} documents indexed in the '{index_name}' index.")
    else:
        print("No .txt files found in the directory.")

In [167]:
# Index documents from the 'files' directory
bulk_index("files", "index_recipes")

5939 documents indexed in the 'index_recipes' index.


In [168]:
def search(query):
   res = es.search(index='index_recipes', body=query)
   print("Search Results:")
   for i, hit in enumerate(res['hits']['hits']):
    res_doc = hit['_source']
    print(f"{i+1}. Title: {res_doc['title']}\n   Content: {res_doc['content']}\n")

In [ ]:
# search({"query": {"match_all": {}}})
# search({"query": {"match": {"title": "burro e salvia"}}})
# search({"query": {"match": {"content": "banane"}}})
# search({"query": {"multi_match": {"query": "burro", "fields": ["title","content"]}}})
# search({"query": {"match_phrase": {"content": "dalla"}}})

Search Results:
1. Title: Smoothie di mele, carote e zenzero
   Content: Per preparare lo smoothie di mele, carote e zenzero ponete in freezer la banana e lasciate che si congeli. Intanto lavate le mele, tagliatele a metà eliminate il torsolo e tagliatele a pezzi Ponetele in una ciotola e irrorate con del succo di arancia Lavate, poi eliminate le estremità della carota tagliatela a tocchi grossolani. Sbucciate la radice di zenzero Sbucciate la banana, oramai congelata e tagliatela a pezzi Ora nel bicchiere di un frullatore versate le mele con il succo d'arancia le carote, la banana, lo yogurt Frullate il tutto In una padella versate le mandorle a lamelle, i semi di chia i corn flakes Unite anche lo zucchero moscovado Versate il mix in una ciotola per far raffreddare Per ultimo, tagliate 100 g di mela a pezzetti, irroratela con del succo di limone per non farla annerire. Quindi versate lo smoothie in 4 ciotole Il vostro smoothie di mele, carote e zenzero è pronto per essere gustato 

2.

In [171]:
# Ask to user to insert a query
query = input("Enter your search query: ")

# Parsing the user input into a match query for content field
type_query = query.split(":", 1)[0]
content_query = query.split(":", 1)[1].strip()

if type_query == "title":
    if content_query.startswith('"') and content_query.endswith('"'):
        phrase = content_query.strip('"')
        search({"query": {"match_phrase": {"title": phrase}}})
    else:
        search({"query": {"match": {"title": content_query}}})
elif type_query == "content":
    if content_query.startswith('"') and content_query.endswith('"'):
        phrase = content_query.strip('"')
        search({"query": {"match_phrase": {"content": phrase}}})
    else:
        search({"query": {"match": {"content": content_query}}})

Search Results:
1. Title: Spaghetti alla Carbonara
   Content: Per preparare gli spaghetti alla carbonara cominciate mettendo sul fuoco una pentola con l’acqua salata per cuocere la pasta. Nel frattempo eliminate la cotenna dal guanciale La cotenna avanzata potrà essere riutilizzata per insaporire altre preparazioni. Versate i pezzetti in una padella antiaderente Nel frattempo tuffate gli spaghetti nell’acqua bollente aggiungete anche la maggior parte del Pecorino previsto dalla ricetta Insaporite con il pepe nero amalgamate tutto con una frusta a mano Aggiungete un cucchiaio di acqua di cottura per diluire il composto e mescolate Intanto il guanciale sarà giunto a cottura, spegnete il fuoco e tenetelo da parte Scolate la pasta al dente direttamente nel tegame con il guanciale Per renderla ben cremosa, al bisogno, potete aggiungere poca acqua di cottura della pasta Servite subito gli spaghetti alla carbonara insaporendoli con il Pecorino avanzato

